In [1]:
# !pip install databricks-sql-connector

In [2]:
import os
from databricks import sql as dbsql 
import pandas as pd 

# load env
from dotenv import load_dotenv
load_dotenv()

# Configuration variables - from your keyvault and Databricks 
databricks_pat = os.getenv("DATABRICKS_PAT")
server_hostname = os.getenv("DATABRICKS_SERVER")
http_path = os.getenv("DATABRICKS_HTTP_PATH")


In [ ]:
# Define the connection to the SQL endpoint
connection = dbsql.connect(
    server_hostname=server_hostname, http_path=http_path, access_token=databricks_pat
)

In [ ]:
# run a query to count the number of rows in the table bronze.loan_data

sql_query = "SELECT COUNT(*) FROM bronze.tradb_tours"

with connection.cursor() as cursor:
# Get the data
     cursor.execute(sql_query)
     data = cursor.fetchall()

In [ ]:
# run a query on the tradb_tours to get the group by on finalDisposition
sql_query = """
SELECT TourCorpWeek, COUNT(*), 
    SUM(CASE WHEN finalDisposition = 'Sale' OR trialDisposition = 'Sale' 
          THEN salePrice ELSE 0 END) AS revenue 
FROM bronze.tradb_tours 
WHERE TourCorpWeek > 3830 
      AND finalDisposition IN ('Sale', 'No Sale', 'Pender', 'No Tour') 
GROUP BY TourCorpWeek
"""

with connection.cursor() as cursor:
    # Get the data
    cursor.execute(sql_query)
    data = cursor.fetchall()

# Convert the data to a pandas dataframe
df = pd.DataFrame(data, columns=["TourCorpWeek", "Count", "Revenue"])
df